In [ ]:
!pip install haystack-ai "transformers[torch,sentencepiece]"
!pip install pdfplumber
!pip install peft
!pip install youtube-transcript-api
!pip install pytube
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.1/372.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 M

In [ ]:
import pdfplumber
with pdfplumber.open("/content/Resume_Soham_Y.pdf") as pdf:
    for page in pdf.pages:
        text += page.extract_text()
        #print(text)


In [ ]:
import heapq
from pytube import extract
from heapq import nlargest
from youtube_transcript_api import YouTubeTranscriptApi
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation

In [ ]:
url = 'https://youtu.be/cUBz04LlLVk?si=UbT2fsPZIvs_TXha'
video_id = extract.video_id(url)
transcript = YouTubeTranscriptApi.get_transcript(video_id)
text = ""
for elem in transcript:
    text = text + " " + elem["text"]


In [ ]:
import spacy
from collections import Counter
import random

# English tokenizer
nlp_model = spacy.load("en_core_web_sm")
def create_mcqs(input_text, question_count=5):
    processed_doc = nlp_model(input_text)
    sentence_list = [sentence.text for sentence in processed_doc.sents]
    selected_sentences = random.sample(sentence_list, min(question_count, len(sentence_list)))
    mcq_list = []
    for sentence in selected_sentences:
        sentence_doc = nlp_model(sentence)
        noun_list = [token.text for token in sentence_doc if token.pos_ == "NOUN"]
        if len(noun_list) < 2:
            continue
        noun_frequency = Counter(noun_list)
        if noun_frequency:
            main_noun = noun_frequency.most_common(1)[0][0]
            question_format = sentence.replace(main_noun, "__________")
            choices = [main_noun]
            for _ in range(3):
                distractor = random.choice(list(set(noun_list) - set([main_noun])))
                choices.append(distractor)

            random.shuffle(choices)

            correct_option = chr(64 + choices.index(main_noun) + 1)
            mcq_list.append((question_format, choices, correct_option))

    return mcq_list


In [ ]:
mcqs_tech = create_mcqs(text)

# Prin generat MCQ
for i, mcq in enumerate(mcqs_tech, start=1):
    question_stem, answer_choices, correct_answer = mcq
    print(f"Q{i}: {question_stem}?")
    for j, choice in enumerate(answer_choices, start=1):
        print(f"{chr(64+j)}: {choice}")
    print(f"Correct Answer: {correct_answer}\n")

In [ ]:

from haystack import Document
from haystack.components.readers import ExtractiveReader

docs = [
    #Document(content=text),
    Document(content="Soham is tom's step-Dad"),
    Document(content="mom and dad of anyone are husband and wife to each other"),
    Document(content="lala is toms's mom")
]

reader = ExtractiveReader(model="deepset/roberta-base-squad2")
reader.warm_up()

question = "who is john"

result = reader.run(query=question, documents=docs)
print(result)


{'answers': [ExtractedAnswer(query='who is john', score=0.6810122132301331, data='niggerr', document=Document(id=24349c1a9146ee9872601803d7679d41e7c99cfb7995721414e0856302c0ad61, content: 'niggerr is also called as john'), context=None, document_offset=ExtractedAnswer.Span(start=0, end=7), context_offset=None, meta={}), ExtractedAnswer(query='who is john', score=0.4044201374053955, data='mom and dad of anyone are husband and wife to each other', document=Document(id=a8c5dab94749bd5626e4425831eb7b0b5f1c9305c0f815b26f5398c4bd8e3f9e, content: 'mom and dad of anyone are husband and wife to each other'), context=None, document_offset=ExtractedAnswer.Span(start=0, end=56), context_offset=None, meta={}), ExtractedAnswer(query='who is john', score=0.39727866649627686, data='toms', document=Document(id=85f3760a6eb31def5e50913f7eb36302557ae3551a0fc549b6d93737a923d627, content: 'lala is toms's mom'), context=None, document_offset=ExtractedAnswer.Span(start=8, end=12), context_offset=None, meta={}

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model = AutoModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")

model(
    "https://templates.invoicehome.com/invoice-template-us-neat-750px.png",
    "What is the invoice number?"
)
# {'score': 0.9943977, 'answer': 'us-001', 'start': 15, 'end': 15}

"""nlp(
    "https://miro.medium.com/max/787/1*iECQRIiOGTmEFLdWkVIH2g.jpeg",
    "What is the purchase amount?"
)
# {'score': 0.9912159, 'answer': '$1,000,000,000', 'start': 97, 'end': 97}

nlp(
    "https://www.accountingcoach.com/wp-content/uploads/2013/10/income-statement-example@2x.png",
    "What are the 2020 net sales?"
)
# {'score': 0.59147286, 'answer': '$ 3,750', 'start': 19, 'end': 20}"""


ValueError: Unrecognized configuration class <class 'transformers.models.vision_encoder_decoder.configuration_vision_encoder_decoder.VisionEncoderDecoderConfig'> for this kind of AutoModel: AutoModel.
Model type should be one of AlbertConfig, AlignConfig, AltCLIPConfig, ASTConfig, AutoformerConfig, BarkConfig, BartConfig, BeitConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BioGptConfig, BitConfig, BlenderbotConfig, BlenderbotSmallConfig, BlipConfig, Blip2Config, BloomConfig, BridgeTowerConfig, BrosConfig, CamembertConfig, CanineConfig, ChameleonConfig, ChineseCLIPConfig, ChineseCLIPVisionConfig, ClapConfig, CLIPConfig, CLIPVisionConfig, CLIPSegConfig, ClvpConfig, LlamaConfig, CodeGenConfig, CohereConfig, ConditionalDetrConfig, ConvBertConfig, ConvNextConfig, ConvNextV2Config, CpmAntConfig, CTRLConfig, CvtConfig, Data2VecAudioConfig, Data2VecTextConfig, Data2VecVisionConfig, DbrxConfig, DebertaConfig, DebertaV2Config, DecisionTransformerConfig, DeformableD

In [ ]:
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration
import requests
from PIL import Image

processor = Pix2StructProcessor.from_pretrained('google/deplot')
model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')

url = "https://preview.redd.it/gdp-per-capita-of-g20-v0-qdkgzqvv7mlb1.png?auto=webp&s=8186c3a76bf4fd298e2e1a62138093677fa461f8"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(images=image, text="Generate underlying data table of the figure below:", return_tensors="pt")
predictions = model.generate(**inputs, max_new_tokens=512)
print(processor.decode(predictions[0], skip_special_tokens=True))


preprocessor_config.json:   0%|          | 0.00/249 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.62k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/851k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.88k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

Arial.TTF:   0%|          | 0.00/276k [00:00<?, ?B/s]

Entity | Values <0x0A> Argentina | 32.0 <0x0A> Australia | 31.0 <0x0A> Brazil | 29.67 <0x0A> Canada | 22.72 <0x0A> China | 13.72 <0x0A> France | 44.41 <0x0A> Germany | 51.38 <0x0A> India | 26.0 <0x0A> Indonesia | 50.2 <0x0A> Italy | 36.81 <0x0A> Japan | 35.39 <0x0A> Republic of Korea | 33.9 <0x0A> Mexico | 12.67 <0x0A> Russia | 14.40 <0x0A> Saudi Arabia | 29.92 <0x0A> South Africa | 6.49 <0x0A> Turkey | 11.93 <0x0A> United Kingdom | 46.37 <0x0A> United States | 80.03


In [ ]:
!pip install wikipedia-api


In [ ]:
import wikipediaapi

# Define a user agent string that describes your bot
user_agent = "MyWikipediaBot/1.0 (sohamyedgaonkar@gmail.com)"

# Pass the user agent to the Wikipedia class constructor
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent=user_agent # Specify the user agent here
)

page = wiki_wiki.page("Python programming")

if page.exists():
    print(f"Title: {page.title}")
    print(page.text)
else:
    print("Page does not exist.")

Title: Python (programming language)
Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically typed and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming. It is often described as a "batteries included" language due to its comprehensive standard library.
Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0. Python 2.0 was released in 2000. Python 3.0, released in 2008, was a major revision not completely backward-compatible with earlier versions. Python 2.7.18, released in 2020, was the last release of Python 2.
Python consistently ranks as one of the most popular programming languages, and has gained widespread use in the machine learning community.

History
Python was 

In [ ]:
import heapq   # Module will be used to generate a summary from tockenized sentences
from pytube import extract # module will be used to interact with youtube Youtube using Video URL
from heapq import nlargest
from youtube_transcript_api import YouTubeTranscriptApi # module will be used to get trasncaript of video
import spacy # module will be used  to build NLP model
from spacy.lang.en.stop_words import STOP_WORDS # module will be used  to build NLP model
from string import punctuation

In [ ]:
url = 'https://youtu.be/cUBz04LlLVk?si=UbT2fsPZIvs_TXha'
video_id = extract.video_id(url)
transcript = YouTubeTranscriptApi.get_transcript(video_id)
text = ""
for elem in transcript:
    text = text + " " + elem["text"]
text

' - [Derek] This is the\nfirst in a series of videos about a chemical reaction\ndiscovered over 125 years ago. It releases a tremendous amount of heat. Oh no, the GoPro. Liquefying metal. It is so hot. It is not an explosive, but it can cause explosions. That is crazy! But the reactants are so inert, they can withstand a\nblow torch indefinitely. Do you think this whole thing\'s going to blow up in my face? In fact, when Hollywood directors wanted something that\nlooked like a nuclear bomb, this is the reaction they chose. But in its most common use, one basically unchanged in over a century, it has helped move billions\nof people all over the world. In the late 1800s, Karl\nand his younger brother, Hans Goldschmidt, were preparing to join\nthe family business. Their father owned a chemical factory making dyes for fabrics. So they studied chemistry\nunder Robert Bunsen of Bunsen burner fame. After their father\'s early death, Karl took over management of the company, later joined by hi

In [ ]:
-----------------------------------------------------------------for mcq generation-----------------------------------------------------------------